In [2]:
# 연재 중 웹툰 크롤링

from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8')
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup

In [4]:
# 연재 중 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

soup = readHtml("http://comic.naver.com/webtoon/weekday.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))

# 모든 요일 링크 추출
for a in a_list:
    src = a['href']
    
    url = urlparse(src)
    toon_id = 'naver_' + str(parse_qs(url.query)['titleId']).replace("['", '').replace("']", '') # ID 생성
    
    toon_url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(toon_url+'1') # 네이버 웹툰 첫페이지
    
    # 요일 추출
    if 'weekday=mon' in toon_url:
        weekday = 'mon'
    if 'weekday=tue' in toon_url:
        weekday = 'tue'
    if 'weekday=wed' in toon_url:
        weekday = 'wed'
    if 'weekday=thu' in toon_url:
        weekday = 'thu'
    if 'weekday=fri' in toon_url:
        weekday = 'fri'
    if 'weekday=sat' in toon_url:
        weekday = 'sat'
    if 'weekday=sun' in toon_url:
        weekday = 'sun'
        
    cursor1.execute(weekday_sql, (toon_id, weekday))
    
    detail = soup.find('div', {'class':'detail'})
    toon_name = detail.find('h2')
    
    wrt_nm = str(toon_name.find('span', {'class':'wrt_nm'}))
    wrt_name = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    toon_name = str(toon_name).replace('\t', '').replace('컷툰', '')
    toon_name = re.sub('<.+?>', '', toon_name, 0).strip()
    terminator = toon_name.index('\n')
    toon_name = toon_name[:terminator].strip('컷') # 제목 추출
    
    # 장르 추출
    genres = str(detail.find('span', {'class':'genre'}))
    genres = re.sub('<.+?>', '', genres, 0).strip()
    genre_list = genres.split(', ')

    desc = str(detail.find('p')).replace('<br/>', '\n')
    toon_desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    toon_thumb_url = soup.find('img')['src'] # 썸네일 주소 추출
    
    # toon_info
    cursor2.execute(sel_sql)
    result = cursor2.fetchall()
    if (toon_id not in result):
        cursor3.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'naver', wrt_name, 'X', toon_thumb_url))
        for genre_name in genre_list:
            print(genre_name)
            cursor4.execute(genre_sql, (toon_id, genre_name))
    db.commit()

스토리
판타지
스토리
드라마
스토리
드라마
스포츠
스토리
일상
드라마
로맨스
스토리
개그
드라마
옴니버스
판타지
드라마
감성
스릴러
스토리
일상
드라마
로맨스
스토리
판타지
액션
스릴러
스토리
액션
스릴러
스토리
판타지
드라마
스릴러
스토리
판타지
액션
시대극
에피소드
드라마
로맨스
감성
스토리
일상
드라마
로맨스
스릴러
스토리
판타지
액션
스토리
일상
드라마
스릴러
스토리
개그
드라마
스토리
일상
드라마
로맨스
스토리
개그
판타지
옴니버스
판타지
스토리
판타지
액션
스토리
판타지
액션
시대극
에피소드
일상
개그
에피소드
일상
개그
판타지
드라마
로맨스
스토리
에피소드
판타지
액션
드라마
스토리
일상
로맨스
스토리
드라마
로맨스
스토리
판타지
액션
드라마
스릴러
에피소드
개그
로맨스
스토리
드라마
스토리
로맨스
스포츠
스토리
판타지
스릴러
스토리
드라마
로맨스
스토리
드라마
시대극
스토리
드라마
스토리
드라마
로맨스
스토리
드라마
스릴러
에피소드
드라마
스릴러
스토리
일상
개그
드라마
에피소드
일상
드라마
감성
스토리
판타지
액션
스토리
액션
드라마
스릴러
스토리
판타지
액션
스토리
액션
드라마
스토리
판타지
액션
에피소드
일상
개그
스토리
로맨스
스토리
일상
드라마
감성
스토리
드라마
로맨스
에피소드
일상
개그
스토리
일상
개그
드라마
옴니버스
판타지
스토리
판타지
액션
스토리
스릴러
스토리
개그
스릴러
스토리
개그
로맨스
스토리
판타지
액션
스토리
판타지
에피소드
판타지
드라마
시대극
에피소드
일상
개그
스토리
일상
드라마
에피소드
일상
개그
에피소드
개그
시대극
스토리
판타지
드라마
스토리
판타지
드라마
로맨스
에피소드
감성
스릴러
스토리
판타지
스토리
일상
드라마
스릴러
스토리
판타지
액션
스토리
개그
판타지
액션
에피소드
일상
드라마
스토리
판타지
액션
스릴러
에피소드
판타지
스릴러
스토리
일상
드라마
감성
스토리
드라마
로맨스


C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_716857' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_716857-에피소드' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_716857-개그' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_716857-로맨스' for key 'PRIMARY'")
  result = self._query(query)


에피소드
개그
로맨스
스토리
판타지
액션
스릴러
스토리
드라마
감성
시대극
스토리
판타지
드라마
스릴러
스토리
드라마
감성
스토리
판타지
스릴러
스토리
액션
스릴러
스토리
개그
판타지
액션
스토리
개그
로맨스
에피소드
일상
스토리
드라마
스토리
판타지
드라마
스토리
드라마
로맨스
감성
스토리
액션
에피소드
옴니버스
일상
판타지
드라마
스토리
일상
드라마
감성
스토리
드라마
로맨스
스토리
드라마
로맨스
스토리
드라마
에피소드
일상
개그
로맨스
스토리
일상
드라마
로맨스
스토리
액션
스토리
판타지
액션
에피소드
일상
드라마
스릴러
스토리
액션
스토리
드라마
로맨스
스토리
드라마
스릴러
스토리
판타지
스토리


C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_723714' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_723714-스토리' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_723714-판타지' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_723714-액션' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_723714-시대극' for key 'PRIMARY'")
  result = self._query(query)


스토리
판타지
액션
시대극
스토리
판타지
드라마
스릴러
스토리
스포츠
스토리
일상


...

개그
스토리
스릴러
스토리
판타지
액션
스토리
액션
스릴러
스토리
판타지
스토리
개그
스릴러
스토리
액션
드라마
스토리
스릴러
스토리
개그
액션
옴니버스
개그
드라마
스토리
액션
스릴러
옴니버스
일상
판타지
드라마
옴니버스
판타지
드라마
스릴러
에피소드
개그
판타지
액션
드라마
스토리
드라마
로맨스
감성
시대극
스토리
판타지
액션
스토리
액션
스릴러
스토리
일상
판타지
스토리
판타지
드라마
로맨스
스토리
드라마
스포츠
스토리
드라마
로맨스
에피소드


C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_732955' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_732955-에피소드' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_732955-일상' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_732955-개그' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_732955-드라마' for key 'PRIMARY'")
  result = self._query(query)


일상
개그
드라마
스토리
판타지
드라마
에피소드


C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_626906' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_626906-에피소드' for key 'PRIMARY'")
  result = self._query(query)
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1062, "Duplicate entry 'naver_626906-일상' for key 'PRIMARY'")
  result = self._query(query)


일상
에피소드
일상
개그
로맨스
스토리
액션
스릴러
에피소드
일상
드라마
감성
스토리
로맨스
감성
스토리
개그
드라마
로맨스
스토리
판타지
로맨스
감성
스토리
드라마
에피소드
스토리
판타지
드라마
로맨스
스토리
판타지
드라마
스토리
드라마
로맨스
스토리
드라마
로맨스
스토리
일상
드라마
로맨스
감성
스토리
개그
스토리
판타지
드라마
로맨스
스토리
드라마
로맨스
에피소드
개그
판타지
액션
스토리
판타지


In [3]:
sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

soup = readHtml("https://comic.naver.com/webtoon/finish.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))
# pprint(a_list)

# 웹툰 정보 추출
for a in a_list:
    src = a['href']
    url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    url = urlparse(src)
    toon_id = 'naver_' + str(parse_qs(url.query)['titleId']).replace("['", '').replace("']", '') # ID 생성

    toon_url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(toon_url+'1') # 네이버 웹툰 첫페이지
    
    detail = soup.find('div', {'class':'detail'})
    toon_name = detail.find('h2')
    
    wrt_nm = str(toon_name.find('span', {'class':'wrt_nm'}))
    wrt_name = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    toon_name = str(toon_name).replace('\t', '').replace('컷툰', '')
    toon_name = re.sub('<.+?>', '', toon_name, 0).strip()
    terminator = toon_name.index('\n')
    toon_name = toon_name[:terminator].strip('컷') # 제목 추출
    
    # 장르 추출
    genres = str(detail.find('span', {'class':'genre'}))
    genres = re.sub('<.+?>', '', genres, 0).strip()
    genre_list = genres.split(', ')

    desc = str(detail.find('p')).replace('<br/>', '\n')
    toon_desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    toon_thumb_url = soup.find('img')['src'] # 썸네일 주소 추출
    
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'naver', wrt_name, 'O', toon_thumb_url))
        for genre_name in genre_list:
            print(genre_name)
            cursor3.execute(genre_sql, (toon_id, genre_name))
    db.commit()

에피소드
일상
개그
스토리
로맨스
스토리
드라마
감성
에피소드
일상
드라마
감성
에피소드
일상
드라마
스토리
개그
로맨스
스토리
드라마
옴니버스
일상
개그
옴니버스
스릴러
스토리
판타지
스토리
드라마
스토리
스릴러
스토리
개그
드라마
스포츠
스토리
스릴러
스토리
스토리
판타지
액션
스토리
옴니버스
개그
스토리
일상
개그
판타지
스토리
드라마
감성
스토리
일상
판타지
드라마
감성
스토리
개그
판타지
에피소드
드라마
감성
스토리
스토리
개그
드라마
로맨스
스토리
드라마
로맨스
감성
스토리
드라마
감성
스토리
판타지
액션
드라마
스토리
드라마
스토리
에피소드
일상
판타지
드라마
에피소드
일상
드라마
감성
스토리
판타지
스토리
판타지
드라마
스토리
드라마
스릴러
스토리
스릴러
스토리
드라마
시대극
스토리
드라마
감성
스토리
드라마
에피소드
일상
개그
스토리
판타지
액션
스토리
개그
판타지
스토리
스릴러
스토리
스토리
드라마
스토리
스포츠
스토리
판타지
드라마
스토리
개그
판타지
스토리
로맨스
스토리
판타지
액션
스토리
판타지
드라마
스토리
판타지
액션
스토리
드라마
로맨스
에피소드
일상
개그
스토리
스토리
로맨스
감성
스토리
판타지
액션
스릴러
스토리
드라마
로맨스
스토리
드라마
스토리
판타지
로맨스
감성
에피소드
일상
개그
드라마
에피소드
일상
드라마
감성
스토리
드라마
로맨스
스릴러
스토리
드라마
에피소드
일상
개그
스토리
판타지
드라마
에피소드
스토리
판타지
드라마
감성
스토리
액션
드라마
시대극
스토리
판타지
드라마
에피소드
일상
로맨스
에피소드
스토리
판타지
드라마
스토리
판타지
액션
드라마
스토리
판타지
에피소드
일상
개그
스토리
스토리
판타지
로맨스
스토리
판타지
액션
드라마
스토리
판타지
액션
스토리
개그
판타지
감성
스토리
일상
판타지
드라마
로맨스
에피소드
옴니버스
일상
개그
판타지
액션
드라마
로맨스
감성
스릴러
시대극
스토리
판타지
액션
드라마
스릴러
에피소드
개그
시대극
스토리
에피소드
스토리
판타지
액션
스토리
드라마
로맨스
스토리
판타지
액